In [54]:
from dotenv import load_dotenv
load_dotenv('.env')

True

In [43]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")

In [44]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "{directive}"),
    ("user", "{input}")
])

In [45]:
llm.invoke("what is langsmith?")

AIMessage(content='Langsmith is a language learning platform that uses artificial intelligence to create personalized language learning experiences. It offers courses in various languages and uses a variety of techniques including interactive exercises, conversation practice and immersion activities to help learners improve their language skills.')

In [46]:
chain = prompt | llm

In [48]:
chain.invoke({
    "input": "what is langsmith?",
    "directive": "You answer in pig latin"
})

AIMessage(content='angSmithLay isay away oolTay atthay anslatestray anguageslay.')

In [23]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [24]:
chain = prompt | llm | output_parser

In [25]:
chain.invoke({"input": "what is langsmith?"})

'Langsmith is an advanced natural language processing (NLP) model developed by OpenAI. It is designed to generate human-like text and assist with various language-related tasks, such as answering questions, providing explanations, and generating coherent and contextually relevant responses. Langsmith utilizes deep learning techniques and has been trained on a vast amount of text data to enhance its language understanding and generation capabilities. It can be a valuable tool for developers, researchers, and anyone interested in exploring and utilizing NLP technology.'

In [27]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [28]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [29]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [30]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [32]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [35]:
response = retrieval_chain.invoke({"input": "what is langsmith?"})
print(response["answer"])

LangSmith is a tool or platform that helps in building reliable LLM (Large Language Model) applications. It provides features such as tracing, debugging, monitoring, and dataset curation to improve the development and performance of LLM applications.


In [49]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [50]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
 Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookRelease NotesOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past

In [51]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [52]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookRelease NotesOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, we at LangChain have been building and using LangSmith with the goal of bridging this gap. This is our tactical user guide to outline effective ways to use LangSmith and maximize its benefits.On by default\u200bAt LangChain, all of us have LangSmith’s tracing running in the background by default. On the Python side, this is 

In [53]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [55]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [56]:
tools = [retriever_tool, search]

In [57]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [58]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
LangSmith can help with testing in several ways:

1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.

2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.

3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been adequately tested and may require additional test cases.

4. Test Prioritization: LangSmith can prioritize the exec

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in several ways:\n\n1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.\n\n2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.\n\n3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been adequately tested and may require additional test cases.\n\n4. Test Prioritization: Lan

In [59]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "Weather in San Francisco is {'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1707988594, 'localtime': '2024-02-15 1:16'}, 'current': {'last_updated_epoch': 1707988500, 'last_updated': '2024-02-15 01:15', 'temp_c': 11.7, 'temp_f': 53.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 230, 'wind_dir': 'SW', 'pressure_mb': 1021.0, 'pressure_in': 30.16, 'precip_mm': 0.03, 'precip_in': 0.0, 'humidity': 89, 'cloud': 75, 'feelslike_c': 10.1, 'feelslike_f': 50.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 12.9, 'gust_kph': 20.8}}"}, {'url': 'https

{'input': 'what is the weather in SF?',
 'output': 'The current weather in San Francisco is partly cloudy with a temperature of 53.1°F (11.7°C). The wind is coming from the southwest at 6.8 km/h. The humidity is 89% and there is a 75% cloud cover.'}

In [60]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...
LangSmith can help test your LLM applications in the following ways:

1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This includes checking for clarity, coherence, grammar, and overall impact.

2. Resume/CV Review: LangSmith can review your resume or CV to ensure it is well-structured, highlights your relevant experiences and achievements, and adheres to the standard format for LLM applications.

3. Recommendation Letter Guidance: LangSmith can provide guidance on selecting appropriate recommenders, as well as tips on how to request strong recommendation letters. Additionally, LangSmith can review draft recommendation letters to ensure they effectively highlight your strengths and qualifications.

4. Application Strategy: LangSmith can help you develop a comprehensive application strategy, including selecting th

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith can help test your LLM applications in the following ways:\n\n1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This includes checking for clarity, coherence, grammar, and overall impact.\n\n2. Resume/CV Review: LangSmith can review your resume or CV to ensure it is well-structured, highlights your relevant experiences and achievements, and adheres to the standard format for LLM applications.\n\n3. Recommendation Letter Guidance: LangSmith can provide guidance on selecting appropriate recommenders, as well as tips on how to request strong recommendation letters. Additionally, LangSmith can review draft recommendation letters to ensure they effectively highlight your strengths and qualificatio